In [1]:
#%load_ext rpy2.ipython
%matplotlib inline
import logging
logging.getLogger('fbprophet').setLevel(logging.ERROR)
import warnings
warnings.filterwarnings("ignore")

## Python API

Prophet follows the `sklearn` model API.  We create an instance of the `Prophet` class and then call its `fit` and `predict` methods.  

The input to Prophet is always a dataframe with two columns: `ds` and `y`.  The `ds` (datestamp) column must contain a date or datetime (either is fine). The `y` column must be numeric, and represents the measurement we wish to forecast.

As an example, let's look at a time series of daily page views for the Wikipedia page for [Peyton Manning](https://en.wikipedia.org/wiki/Peyton_Manning).  We scraped this data using the [Wikipediatrend](https://cran.r-project.org/web/packages/wikipediatrend/vignettes/using-wikipediatrend.html) package in R.  Peyton Manning provides a nice example because it illustrates some of Prophet's features, like multiple seasonality, changing growth rates, and the ability to model special days (such as Manning's playoff and superbowl appearances). The CSV is available [here](https://github.com/facebook/prophet/blob/master/examples/example_wp_peyton_manning.csv).

First we'll import the data and log-transform the y variable.

In [2]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import time

In [3]:
pd.options.display.max_rows = 10

In [4]:
df_bitcoin       = pd.read_csv('data/dataset_20180529_Hourly.csv')
df_bitcoin['ds'] = pd.to_datetime(df_bitcoin['Timestamp'],unit='s')
df_bitcoin['y']  = np.log(df_bitcoin['Close'])

df = pd.DataFrame(df_bitcoin[['ds', 'y']])
df

,ds,y
0,2011-12-31 07:00:00,1.479329
1,2011-12-31 08:00:00,709.726837
2,2011-12-31 09:00:00,709.726837
3,2011-12-31 10:00:00,709.726837
4,2011-12-31 11:00:00,709.726837
...,...,...
56214,2018-05-29 03:00:00,8.871430
56215,2018-05-29 04:00:00,8.872682
56216,2018-05-29 05:00:00,8.868073
56217,2018-05-29 06:00:00,8.869101


In [5]:
df = df.set_index('ds')

In [6]:
# check anomally
df = df.drop(df.index[np.abs(df.y)> 50])

In [7]:
df = df.reset_index()

In [8]:
#df = df[df.ds > '2015-01-01']
df

,ds,y
0,2011-12-31 07:00:00,1.479329
1,2011-12-31 15:00:00,1.479329
2,2011-12-31 16:00:00,1.519513
3,2011-12-31 17:00:00,1.521699
4,2012-01-01 04:00:00,1.521699
...,...,...
52721,2018-05-29 03:00:00,8.871430
52722,2018-05-29 04:00:00,8.872682
52723,2018-05-29 05:00:00,8.868073
52724,2018-05-29 06:00:00,8.869101


We fit the model by instantiating a new `Prophet` object.  Any settings to the forecasting procedure are passed into the constructor.  Then you call its `fit` method and pass in the historical dataframe. Fitting should take 1-5 seconds.

In [9]:
m = Prophet()
m.fit(df);

Predictions are then made on a dataframe with a column `ds` containing the dates for which a prediction is to be made. You can get a suitable dataframe that extends into the future a specified number of days using the helper method `Prophet.make_future_dataframe`. By default it will also include the dates from the history, so we will see the model fit as well. 

In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
53086,2019-05-25 07:00:00
53087,2019-05-26 07:00:00
53088,2019-05-27 07:00:00
53089,2019-05-28 07:00:00
53090,2019-05-29 07:00:00


The `predict` method will assign each row in `future` a predicted value which it names `yhat`.  If you pass in historical dates, it will provide an in-sample fit. The `forecast` object here is a new dataframe that includes a column `yhat` with the forecast, as well as columns for components and uncertainty intervals.

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast

You can plot the forecast by calling the `Prophet.plot` method and passing in your forecast dataframe.

In [ ]:
m.plot(forecast);

If you want to see the forecast components, you can use the `Prophet.plot_components` method.  By default you'll see the trend, yearly seasonality, and weekly seasonality of the time series.  If you include holidays, you'll see those here, too.

In [ ]:
m.plot_components(forecast);

More details about the options available for each method are available in the docstrings, for example, via `help(Prophet)` or `help(Prophet.fit)`.